In [1]:
import csv
import pickle

In [2]:
from math import log10
from collections import defaultdict

In [3]:
univ_final_comment = defaultdict(lambda: [])

with open('./report_data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for data in reader:
        univ_final_comment[data['university']].append(data['final_comment'])
        
del univ_final_comment['']

In [4]:
univ_final_comment

defaultdict(<function __main__.<lambda>()>,
            {'Leuphana University of Lüneburg': ['정말 너무너무 행복한 6개월이었어요. 원래 교환학생에 대한 로망도 없었고 가고 싶은 생각도 없었는데 간거였는데도 너무 만족했습니다. 학교 수업 같은 경우는 만족스러운 수업은 한 개밖에 없긴 했지만 저는 공부가 그렇게 큰 목적은 아니였어서..ㅎ 그냥 한국 수업 같은 느낌이었구요,  저는 시간표가 진짜 이래도 되나 싶을 정도로 많이 널널했어서 여행을 정말 많이 다녔습니다.(다시 한번 강조하지만 미컴과시라면 로이파나로 오세요!) 이주 여행하고 이주 쉬고 다시 여행하고 쉬고의 반복이었어요. 중간에는 엄마랑 오빠도 와서 3주동안 같이 여행했습니다! 저희 학교가 독일이어도 북쪽이라 위치가 엄청나게 좋다고 보기는 힘들어요. 독일 중에서 여행 다니기 좋은 도시는 베를린이에요! 비행기 값이 훨씬 싸더라구요! 학교 선택하실 때 참고하세요! 저는 그래서 일부러 베를린까지 플릭스 버스 타고 비행기는 베를린에서 타기도 했어요. 학교 생활은 국제처에서 프로그램들을 많이 제공해서 다른 나라 친구들과 친해질 기회를 많이 만들어줘요! 열심히 참여하시면 친해지실 수 있습니다. 독일은 파티가 정말 많아요. 근데 초대받은 사람만 가는게 아니라 파티가 있으면 가고 싶은 사람들은 그냥 막 가면 되는 문화라 그런 파티에 많이 가시면 외국인 친구들 많이 사귈 수 있습니다! 근데 언어가 생각보다 스트레스에요. 친해지는 건 가능해도 언어가 딸려서 깊은 대화가 불가능해서 한계를 많이 느꼈습니다. 저는 오시기 전에 괜히 독일어 공부하는 것 보다 영어 회화 공부 하시는걸 추천드려요. 한국인 빼고는 영어를 다 잘해서 자괴감 많이 들어요..ㅎㅎ 그래서 한국인들끼리 더 돈독해지기도 하는데요, 저희는 한국인이 10명이었는데 정말 너무 잘 지내서 행복했어요. 서로 집에 초대해서 한식 거하게 해먹고 같이 수영장도 가고 초등학교 때 동네 친구들이랑 노는 느낌

In [5]:
def pickling(title, dic):
    f = open("{}.txt".format(title), 'wb')
    pickle.dump(dic, f)
    f.close

In [6]:
from konlpy.tag import Okt

D는 문서 집합, d는 문서 하나, t는 단어  
하지만 문서 집합(대학) 수준에서 키워드를 추출하고자 하기 때문에  
D는 전체 문서 집합, d는 문서 집합, r은 문서, t는 단어  
가 된다. 이때 idf는 유지한다.

TF

In [7]:
def tf_btf(t, d):
    tf = 0
    for r in d:
        tf += r.count(t)  
    return tf

def tf_ntf1(t, d):
    max_btf = -1
    for r in d:
        for term in r:
            if tf_btf(term, d) > max_btf:
                max_btf = tf_btf(term, d)
    btf = tf_btf(t, d)
    return btf / max_btf

def tf_ntf2(t, d):
    tf = 0
    for r in d:
        denominator = 0
        for term in r:
            denominator += r.count(term)
        nominator = r.count(t)
        if denominator != 0:
            tf += nominator / denominator
    return tf

IDF

In [8]:
def idf(t, d):
    numerator = len(d)
    denominator = len([True for r in d if t in r])
    return log10(numerator/denominator)

TF-IDF

In [9]:
def tfidf_btf(t, d):
    return tf_btf(t, d) * idf(t, d)

def tfidf_ntf1(t, d):
    return tf_ntf1(t, d) * idf(t, d)

def tfidf_ntf2(t, d):
    return tf_ntf2(t, d) * idf(t, d)

def tfidf_btf_log(t, d):
    return (log10(tf_btf(t, d)) + 1) * idf(t, d)

def tfidf_ntf1_log(t, d):
    return (log10(tf_ntf1(t, d)) + 1) * idf(t, d)

def tfidf_ntf2_log(t, d):
    return (log10(tf_ntf2(t, d)) + 1) * idf(t, d)

In [10]:
def okt_tokenize(d):
    okt = Okt()
    tokens = okt.pos(d, stem=True) # 단어 기준
    stopword = ['아니다', '되어다', '않다', '있다', '없다', '이다', '개월', '되다', '같다', 
                '하다', '다른', '성균관대', '성균관', '성대'] # 불용어
    result_tokens = []
    for token in tokens: # 명사, 동사, 형용사만 키워드로 포함
        if ((token[0] not in stopword) 
            and ((token[1] == 'Noun' and len(token[0]) > 1) 
                 or (token[1] == 'Verb' and len(token[0]) > 2) 
                 or (token[1] == 'Adjective' and len(token[0]) > 2))):
            result_tokens.append(token[0])
    return result_tokens

def tfidfScorer(d, version):
    tokenized_d = [okt_tokenize(r) for r in d] # 토큰화 후 TF-IDF 계산
    result = []
    for tokenized_r in tokenized_d:
        if version == 0:
            result.append([(t, tfidf_btf(t, tokenized_d)) for t in tokenized_r])
        if version == 1:
            result.append([(t, tfidf_ntf1(t, tokenized_d)) for t in tokenized_r])
        if version == 2:
            result.append([(t, tfidf_ntf2(t, tokenized_d)) for t in tokenized_r])
        if version == 3:
            result.append([(t, tfidf_btf_log(t, tokenized_d)) for t in tokenized_r])
        if version == 4:
            result.append([(t, tfidf_ntf1_log(t, tokenized_d)) for t in tokenized_r])
        if version == 5:
            result.append([(t, tfidf_ntf2_log(t, tokenized_d)) for t in tokenized_r])
    return result

In [11]:
univ_tfidf_btf = {} # 0
univ_tfidf_ntf1 = {} # 1
univ_tfidf_ntf2 = {} # 2
univ_tfidf_btf_log = {} # 3
univ_tfidf_ntf1_log = {} # 4
univ_tfidf_ntf2_log = {} # 5

for idx, key in enumerate(univ_final_comment.keys()):
    print('키워드 추출 중 ... {}'.format(idx + 1))
    corpus = univ_final_comment[key]
    temp = []
    for s in tfidfScorer(corpus, 0):
        temp += s[:10]
    univ_tfidf_btf[key] = sorted(set(temp), key=lambda x:x[1], reverse=True)[:100]
    temp = []
    for s in tfidfScorer(corpus, 1):
        temp += s[:10]
    univ_tfidf_ntf1[key] = sorted(set(temp), key=lambda x:x[1], reverse=True)[:100]
    temp = []
    for s in tfidfScorer(corpus, 2):
        temp += s[:10]
    univ_tfidf_ntf2[key] = sorted(set(temp), key=lambda x:x[1], reverse=True)[:100]
    temp = []
    for s in tfidfScorer(corpus, 3):
        temp += s[:10]
    univ_tfidf_btf_log[key] = sorted(set(temp), key=lambda x:x[1], reverse=True)[:100]
    temp = []
    for s in tfidfScorer(corpus, 4):
        temp += s[:10]
    univ_tfidf_ntf1_log[key] = sorted(set(temp), key=lambda x:x[1], reverse=True)[:100]
    temp = []
    for s in tfidfScorer(corpus, 5):
        temp += s[:10]
    univ_tfidf_ntf2_log[key] = sorted(set(temp), key=lambda x:x[1], reverse=True)[:100]

키워드 추출 중 ... 1
키워드 추출 중 ... 2
키워드 추출 중 ... 3
키워드 추출 중 ... 4
키워드 추출 중 ... 5
키워드 추출 중 ... 6
키워드 추출 중 ... 7
키워드 추출 중 ... 8
키워드 추출 중 ... 9
키워드 추출 중 ... 10
키워드 추출 중 ... 11
키워드 추출 중 ... 12
키워드 추출 중 ... 13
키워드 추출 중 ... 14
키워드 추출 중 ... 15
키워드 추출 중 ... 16
키워드 추출 중 ... 17
키워드 추출 중 ... 18
키워드 추출 중 ... 19
키워드 추출 중 ... 20
키워드 추출 중 ... 21
키워드 추출 중 ... 22
키워드 추출 중 ... 23
키워드 추출 중 ... 24
키워드 추출 중 ... 25
키워드 추출 중 ... 26
키워드 추출 중 ... 27
키워드 추출 중 ... 28
키워드 추출 중 ... 29
키워드 추출 중 ... 30
키워드 추출 중 ... 31
키워드 추출 중 ... 32
키워드 추출 중 ... 33
키워드 추출 중 ... 34
키워드 추출 중 ... 35
키워드 추출 중 ... 36
키워드 추출 중 ... 37
키워드 추출 중 ... 38
키워드 추출 중 ... 39
키워드 추출 중 ... 40
키워드 추출 중 ... 41
키워드 추출 중 ... 42
키워드 추출 중 ... 43
키워드 추출 중 ... 44
키워드 추출 중 ... 45
키워드 추출 중 ... 46
키워드 추출 중 ... 47
키워드 추출 중 ... 48
키워드 추출 중 ... 49
키워드 추출 중 ... 50
키워드 추출 중 ... 51
키워드 추출 중 ... 52
키워드 추출 중 ... 53
키워드 추출 중 ... 54
키워드 추출 중 ... 55
키워드 추출 중 ... 56
키워드 추출 중 ... 57
키워드 추출 중 ... 58
키워드 추출 중 ... 59
키워드 추출 중 ... 60
키워드 추출 중 ... 61
키워드 추출 중 ... 62
키워드 추출 중 ... 63
키

In [14]:
univ_tfidf_btf

{'Leuphana University of Lüneburg': [('학교', 3.1126050015345745),
  ('수업', 2.3344537511509307),
  ('여행', 2.113095108668175),
  ('느끼다', 2.1072099696478683),
  ('공부', 1.9084850188786497),
  ('재미', 1.5563025007672873),
  ('함부르크', 1.5563025007672873),
  ('생각', 1.4313637641589874),
  ('정말', 1.266899936761997),
  ('지내다', 1.2041199826559248),
  ('사람', 1.0565475543340874),
  ('엄청나다', 0.9542425094393249),
  ('교환학생', 0.9542425094393249),
  ('마음껏', 0.9542425094393249),
  ('다가', 0.9542425094393249),
  ('학기', 0.9542425094393249),
  ('느낌', 0.9542425094393249),
  ('때문', 0.9542425094393249),
  ('위치', 0.9542425094393249),
  ('자체', 0.9030899869919435),
  ('여전하다', 0.9030899869919435),
  ('예쁘다', 0.9030899869919435),
  ('도시', 0.9030899869919435),
  ('목표', 0.7781512503836436),
  ('대한', 0.7781512503836436),
  ('하루', 0.7781512503836436),
  ('만족하다', 0.7781512503836436),
  ('대학도시', 0.7781512503836436),
  ('로망', 0.7781512503836436),
  ('어우러지다', 0.7781512503836436),
  ('듯이', 0.7781512503836436),
  ('필요하다', 0.77815

In [15]:
univ_tfidf_ntf1

{'Leuphana University of Lüneburg': [('학교', 0.1945378125959109),
  ('수업', 0.14590335944693317),
  ('여행', 0.13206844429176093),
  ('느끼다', 0.13170062310299177),
  ('공부', 0.11928031367991561),
  ('함부르크', 0.09726890629795545),
  ('재미', 0.09726890629795545),
  ('생각', 0.08946023525993671),
  ('정말', 0.07918124604762482),
  ('지내다', 0.0752574989159953),
  ('사람', 0.06603422214588046),
  ('위치', 0.059640156839957804),
  ('교환학생', 0.059640156839957804),
  ('다가', 0.059640156839957804),
  ('엄청나다', 0.059640156839957804),
  ('느낌', 0.059640156839957804),
  ('마음껏', 0.059640156839957804),
  ('때문', 0.059640156839957804),
  ('학기', 0.059640156839957804),
  ('예쁘다', 0.05644312418699647),
  ('도시', 0.05644312418699647),
  ('자체', 0.05644312418699647),
  ('여전하다', 0.05644312418699647),
  ('만족하다', 0.04863445314897773),
  ('휴가', 0.04863445314897773),
  ('대학도시', 0.04863445314897773),
  ('어우러지다', 0.04863445314897773),
  ('목표', 0.04863445314897773),
  ('갖추다', 0.04863445314897773),
  ('로망', 0.04863445314897773),
  ('하루', 

In [16]:
univ_tfidf_ntf2

{'Leuphana University of Lüneburg': [('마음껏', 0.06361616729595498),
  ('다가', 0.06361616729595498),
  ('함부르크', 0.059857788491049516),
  ('예쁘다', 0.05171540951150446),
  ('자체', 0.04640879099819711),
  ('여전하다', 0.04640879099819711),
  ('지내다', 0.04145187422097906),
  ('재미', 0.03383266306015842),
  ('사람', 0.031903933701955836),
  ('여행', 0.030767213973857652),
  ('갖추다', 0.029928894245524758),
  ('어우러지다', 0.029928894245524758),
  ('필요하다', 0.029928894245524758),
  ('자연', 0.029928894245524758),
  ('근교', 0.029928894245524758),
  ('대학도시', 0.029928894245524758),
  ('뤼네부르크', 0.02753188116931348),
  ('정말', 0.022053400192029955),
  ('느끼다', 0.021988801855605422),
  ('느낌', 0.019392566953671198),
  ('위치', 0.019392566953671198),
  ('도시', 0.018506805732197366),
  ('어떻다', 0.01691633153007921),
  ('하루', 0.01691633153007921),
  ('놀수', 0.01691633153007921),
  ('목표', 0.016211484382992573),
  ('듯이', 0.016211484382992573),
  ('휴가', 0.016211484382992573),
  ('행복하다', 0.015995541270707098),
  ('공부', 0.013497449582918

In [17]:
univ_tfidf_btf_log

{'Leuphana University of Lüneburg': [('학교', 1.2466449854414634),
  ('수업', 1.1494237513283618),
  ('재미', 1.0123981179125534),
  ('함부르크', 1.0123981179125534),
  ('목표', 0.7781512503836436),
  ('대한', 0.7781512503836436),
  ('하루', 0.7781512503836436),
  ('만족하다', 0.7781512503836436),
  ('대학도시', 0.7781512503836436),
  ('로망', 0.7781512503836436),
  ('어우러지다', 0.7781512503836436),
  ('듯이', 0.7781512503836436),
  ('필요하다', 0.7781512503836436),
  ('자연', 0.7781512503836436),
  ('갖추다', 0.7781512503836436),
  ('원래', 0.7781512503836436),
  ('놀수', 0.7781512503836436),
  ('휴가', 0.7781512503836436),
  ('어떻다', 0.7781512503836436),
  ('근교', 0.7781512503836436),
  ('공부', 0.764376873198569),
  ('생각', 0.7047659464249274),
  ('엄청나다', 0.6207490639591157),
  ('교환학생', 0.6207490639591157),
  ('마음껏', 0.6207490639591157),
  ('다가', 0.6207490639591157),
  ('학기', 0.6207490639591157),
  ('느낌', 0.6207490639591157),
  ('때문', 0.6207490639591157),
  ('위치', 0.6207490639591157),
  ('느끼다', 0.555429854985112),
  ('지내다', 0.482268

In [18]:
univ_tfidf_ntf1_log

{'Leuphana University of Lüneburg': [('학교', 0.30965751532582403),
  ('수업', 0.21243628121272276),
  ('느끼다', 0.19295362182728582),
  ('공부', 0.18986563624075592),
  ('여행', 0.15409063962149958),
  ('생각', 0.13025470946711445),
  ('지내다', 0.11979187908506812),
  ('사람', 0.10108188867150285),
  ('자체', 0.0821815717456083),
  ('여전하다', 0.0821815717456083),
  ('예쁘다', 0.0821815717456083),
  ('도시', 0.0821815717456083),
  ('정말', 0.07918124604762482),
  ('함부르크', 0.07541064779691431),
  ('재미', 0.07541064779691431),
  ('뤼네부르크', 0.07007375715568781),
  ('행복하다', 0.05075344127081867),
  ('다가', 0.04623782700130271),
  ('엄청나다', 0.04623782700130271),
  ('마음껏', 0.04623782700130271),
  ('때문', 0.04623782700130271),
  ('교환학생', 0.04623782700130271),
  ('느낌', 0.04623782700130271),
  ('위치', 0.04623782700130271),
  ('학기', 0.04623782700130271),
  ('목표', -0.15883621973199552),
  ('대한', -0.15883621973199552),
  ('하루', -0.15883621973199552),
  ('만족하다', -0.15883621973199552),
  ('대학도시', -0.15883621973199552),
  ('로망', -0.1

In [19]:
univ_tfidf_ntf2_log

{'Leuphana University of Lüneburg': [('예쁘다', 0.0707451181848341),
  ('다가', 0.05961092677364147),
  ('마음껏', 0.05961092677364147),
  ('자체', 0.05659077408831247),
  ('여전하다', 0.05659077408831247),
  ('사람', 0.04545028719136376),
  ('여행', 0.04267578235050934),
  ('지내다', 0.04182342163157302),
  ('정말', 0.035224032794853595),
  ('뤼네부르크', 0.03417897665060979),
  ('행복하다', 0.02418010040520258),
  ('느끼다', -0.04106296182461488),
  ('도시', -0.06360110759656948),
  ('함부르크', -0.08866516207046903),
  ('느낌', -0.1865512922884482),
  ('위치', -0.1865512922884482),
  ('공부', -0.261642346129324),
  ('재미', -0.28147896789565946),
  ('생각', -0.2856033281659817),
  ('교환학생', -0.29638427539101647),
  ('엄청나다', -0.3043824619034274),
  ('학기', -0.3043824619034274),
  ('때문', -0.3043824619034274),
  ('대학도시', -0.32291202959937887),
  ('어우러지다', -0.32291202959937887),
  ('필요하다', -0.32291202959937887),
  ('자연', -0.32291202959937887),
  ('갖추다', -0.32291202959937887),
  ('근교', -0.32291202959937887),
  ('하루', -0.5157258354245693),


In [20]:
pickling('univ_tfidf_btf', univ_tfidf_btf)
pickling('univ_tfidf_ntf1', univ_tfidf_ntf1)
pickling('univ_tfidf_ntf2', univ_tfidf_ntf2)
pickling('univ_tfidf_btf_log', univ_tfidf_btf_log)
pickling('univ_tfidf_ntf1_log', univ_tfidf_ntf1_log)
pickling('univ_tfidf_ntf2_log', univ_tfidf_ntf2_log)

교차비교

In [81]:
def get_keyword(univ_tfidf):
    keywords = []
    for univ in univ_tfidf.keys():
        for word in univ_tfidf[univ]:
            if word not in keywords:
                keywords.append(word[0])
    return keywords

In [82]:
import numpy

def standard_deviation(keywords, univ_tfidf, thres):
    threshold = thres
    for kw in keywords:
        dev_arr = []
        for key in univ_tfidf.keys():
            for tfidf in univ_tfidf[key]: # tfidf (word, tfidf value)
                if kw in tfidf:
                    dev_arr.append(univ_tfidf[key].index(tfidf))
        if len(dev_arr) > 2 and numpy.std(dev_arr) < threshold:
            for key in univ_tfidf.keys():
                for tfidf in univ_tfidf[key]:
                    if kw in tfidf:
                        del univ_tfidf[key][univ_tfidf[key].index(tfidf)]

In [115]:
with open('univ_tfidf_btf_log.txt', 'rb') as file:
    univ_tfidf_btf_log = pickle.load(file)
    
len(univ_tfidf_btf_log['Leuphana University of Lüneburg'])

41

In [116]:
keywords = get_keyword(univ_tfidf_btf_log)
standard_deviation(keywords, univ_tfidf_btf_log, 10)

In [117]:
univ_tfidf_btf_log

{'Leuphana University of Lüneburg': [('학교', 1.2466449854414634),
  ('수업', 1.1494237513283618),
  ('재미', 1.0123981179125534),
  ('함부르크', 1.0123981179125534),
  ('목표', 0.7781512503836436),
  ('대한', 0.7781512503836436),
  ('하루', 0.7781512503836436),
  ('만족하다', 0.7781512503836436),
  ('대학도시', 0.7781512503836436),
  ('로망', 0.7781512503836436),
  ('어우러지다', 0.7781512503836436),
  ('듯이', 0.7781512503836436),
  ('필요하다', 0.7781512503836436),
  ('갖추다', 0.7781512503836436),
  ('원래', 0.7781512503836436),
  ('놀수', 0.7781512503836436),
  ('휴가', 0.7781512503836436),
  ('근교', 0.7781512503836436),
  ('공부', 0.764376873198569),
  ('생각', 0.7047659464249274),
  ('엄청나다', 0.6207490639591157),
  ('교환학생', 0.6207490639591157),
  ('마음껏', 0.6207490639591157),
  ('다가', 0.6207490639591157),
  ('학기', 0.6207490639591157),
  ('때문', 0.6207490639591157),
  ('느끼다', 0.555429854985112),
  ('지내다', 0.4822681122428943),
  ('자체', 0.44465780490343443),
  ('여전하다', 0.44465780490343443),
  ('예쁘다', 0.44465780490343443),
  ('도시', 0.4

In [118]:
len(univ_tfidf_btf_log['Leuphana University of Lüneburg'])

37

피클링

In [119]:
for key in univ_tfidf_btf_log.keys():
    univ_tfidf_btf_log[key] = sorted(univ_tfidf_btf_log[key], key=lambda x:(len(x[0]), x[1]), reverse=True)[:10]

[('어우러지다', 0.7781512503836436), ('뤼네부르크', 0.28210876095567466), ('함부르크', 1.0123981179125534), ('만족하다', 0.7781512503836436), ('대학도시', 0.7781512503836436), ('필요하다', 0.7781512503836436), ('엄청나다', 0.6207490639591157), ('교환학생', 0.6207490639591157), ('여전하다', 0.44465780490343443), ('행복하다', 0.14609716188835917)]
[('다양하다', 0.3010299956639812), ('교환학생', 0.3010299956639812), ('유학생', 0.3010299956639812), ('중국어', 0.3010299956639812), ('가능', 0.3010299956639812), ('영어', 0.3010299956639812), ('경험', 0.3010299956639812), ('중국', 0.3010299956639812), ('동시', 0.3010299956639812)]
[('평화롭다', 0.6020599913279624), ('만족하다', 0.6020599913279624), ('지루하다', 0.6020599913279624), ('다이나믹', 0.6020599913279624), ('교환학생', 0.44465780490343443), ('다양하다', 0.39164905395343774), ('보고서', 0.6020599913279624), ('다니다', 0.39164905395343774), ('보름스', 0.18454966338194143), ('파티', 0.7832981079068755)]
[('버킷리스트', 1.2304489213782739), ('만족스럽다', 1.2092019008920885), ('영어실력', 1.2304489213782739), ('불안하다', 1.2304489213782739), ('회복하다', 1.2

In [120]:
univ_tfidf_btf_log

{'Leuphana University of Lüneburg': [('어우러지다', 0.7781512503836436),
  ('뤼네부르크', 0.28210876095567466),
  ('함부르크', 1.0123981179125534),
  ('만족하다', 0.7781512503836436),
  ('대학도시', 0.7781512503836436),
  ('필요하다', 0.7781512503836436),
  ('엄청나다', 0.6207490639591157),
  ('교환학생', 0.6207490639591157),
  ('여전하다', 0.44465780490343443),
  ('행복하다', 0.14609716188835917)],
 'Renmin University of China, Renmin Business School': [('다양하다',
   0.3010299956639812),
  ('교환학생', 0.3010299956639812),
  ('유학생', 0.3010299956639812),
  ('중국어', 0.3010299956639812),
  ('가능', 0.3010299956639812),
  ('영어', 0.3010299956639812),
  ('경험', 0.3010299956639812),
  ('중국', 0.3010299956639812),
  ('동시', 0.3010299956639812)],
 'Hochschule Worms University of Applied Sciences': [('평화롭다',
   0.6020599913279624),
  ('만족하다', 0.6020599913279624),
  ('지루하다', 0.6020599913279624),
  ('다이나믹', 0.6020599913279624),
  ('교환학생', 0.44465780490343443),
  ('다양하다', 0.39164905395343774),
  ('보고서', 0.6020599913279624),
  ('다니다', 0.39164905395343

In [121]:
pickling('result_keyword_renew', univ_tfidf_btf_log)